# Lab 7: P-Tuning - Fine-Tuning a BERT Model for NLU Tasks
---
## Notebook 2: The Training Process

**Goal:** In this notebook, you will fine-tune a BERT model using **P-Tuning** on a classification task. P-Tuning differs from Prompt Tuning by using an MLP-based prompt encoder instead of directly optimizing prompt embeddings.

**You will learn to:**
-   Load a classification dataset and preprocess it for BERT.
-   Configure P-Tuning with virtual tokens and prompt encoder.
-   Apply P-Tuning to the BERT model.
-   Fine-tune the model by training only the prompt encoder parameters.


### Step 1: Load Dataset and Preprocess

We will use the GLUE SST-2 dataset, which is a sentiment classification task. This is ideal for demonstrating P-Tuning on natural language understanding tasks.


### Step 2: Load the Base Model

Next, we load the BERT model for sequence classification. This will be our base model that we'll apply P-Tuning to.


In [ ]:
from transformers import AutoModelForSequenceClassification

# Load base model
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=2  # Binary classification for SST-2
)

print("✅ Base BERT model loaded.")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")


### Step 3: Configure P-Tuning

Now we configure P-Tuning. Unlike Prompt Tuning which directly optimizes prompt embeddings, P-Tuning uses an MLP-based prompt encoder to generate better prompt representations.

#### Key Hugging Face `peft` Components:

-   `peft.PromptTuningConfig`: The configuration class for P-Tuning.
    -   `task_type="SEQ_CLS"`: We specify the task type for sequence classification.
    -   `prompt_tuning_init="RANDOM"`: Random initialization of prompt parameters.
    -   `num_virtual_tokens`: The number of virtual tokens (prompt length).
    -   `prompt_tuning_init_text=None`: P-Tuning doesn't use text initialization.
-   `peft.get_peft_model`: Applies the configuration to our base model.


In [ ]:
from peft import get_peft_model, PromptTuningConfig, TaskType

# --- P-Tuning Configuration ---
# Note: P-Tuning uses PromptTuningConfig but with specific settings
ptuning_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,           # Sequence classification task
    prompt_tuning_init="RANDOM",          # Random initialization
    num_virtual_tokens=10,                # Number of virtual tokens
    prompt_tuning_init_text=None,         # No text initialization for P-Tuning
    tokenizer_name_or_path=model_checkpoint
)

# --- Create PeftModel ---
peft_model = get_peft_model(model, ptuning_config)

# --- Print Trainable Parameters ---
peft_model.print_trainable_parameters()


### Step 4: Set Up Training

The final step is to configure and run the training process using the `transformers.Trainer`.


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# --- Evaluation Function ---
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }

# --- Training Arguments ---
training_args = TrainingArguments(
    output_dir="./bert-ptuning-sst2",
    auto_find_batch_size=True,
    learning_rate=1e-3,                    # Higher learning rate for PEFT
    num_train_epochs=3,
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

# --- Data Collator ---
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# --- Create Trainer ---
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# --- Start Training ---
print("🚀 Starting training with P-Tuning...")
trainer.train()
print("✅ Training complete!")


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load dataset
dataset = load_dataset("glue", "sst2", split="train[:1000]")
dataset = dataset.train_test_split(test_size=0.1)

# Load tokenizer
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Preprocessing function
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets.set_format("torch")

print("✅ Dataset loaded and preprocessed.")
print(tokenized_datasets["train"][0].keys())
